# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
# TODO: Import the necessary libs
# For example: 
# import os

# from lib.agents import Agent
# from lib.llm import LLM
# from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
# from lib.tooling import tool

import os

import chromadb
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

In [4]:
load_dotenv(".env")
os.environ["OPENAI_BASE_URL"] = "https://openai.vocareum.com/v1"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert OPENAI_API_KEY and OPENAI_API_KEY.startswith("voc-"), "Need a voc- OPENAI_API_KEY"
assert TAVILY_API_KEY, "Missing TAVILY_API_KEY"

openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=os.getenv("OPENAI_BASE_URL"))
tavily = TavilyClient(TAVILY_API_KEY)

In [5]:
from lib.voc_embedding import VocareumEmbeddingFunction

embedding_fn = VocareumEmbeddingFunction("text-embedding-3-small")
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay", embedding_function=embedding_fn)

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
def retrieve_game(q, k=3):
    r = collection.query(
        query_texts=[q], n_results=k,
        include=["metadatas","distances"]
    )
    out = []
    for i, m in enumerate(r["metadatas"][0]):
        out.append({
            "name": m.get("Name"),
            "platform": m.get("Platform"),
            "year": m.get("YearOfRelease"),
            "publisher": m.get("Publisher"),
            "score": round(1 - r["distances"][0][i], 4)
        })
    return out

retrieve_game("What platform was Gran Turismo launched on?", 2)


[{'name': 'Gran Turismo',
  'platform': 'PlayStation 1',
  'year': 1997,
  'publisher': 'Sony Computer Entertainment',
  'score': 0.2702},
 {'name': 'Gran Turismo 5',
  'platform': 'PlayStation 3',
  'year': 2010,
  'publisher': 'Sony Computer Entertainment',
  'score': 0.1986}]

#### Evaluate Retrieval Tool

In [7]:
import json

def evaluate_retrieval(question, retrieved_docs):
    docs = "\n".join([f"- {d.get('name')} | {d.get('platform')} | {d.get('year')}"
                      for d in retrieved_docs[:3]])
    ask = (
        "Are these docs enough to answer the question?\n"
        "Reply ONLY as JSON with keys: useful (true/false), description (short).\n\n"
        f"Question: {question}\nDocs:\n{docs}"
    )
    r = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type":"json_object"},
        messages=[{"role":"system","content":"Be brief. JSON only."},
                  {"role":"user","content": ask}]
    )
    return json.loads(r.choices[0].message.content)

# quick test
hits = retrieve_game("What platform was Gran Turismo launched on?", k=3)
evaluate_retrieval("What platform was Gran Turismo launched on?", hits)

{'useful': True,
 'description': 'The documents mention Gran Turismo launched on PlayStation 1 in 1997.'}

#### Game Web Search Tool

In [ ]:
def game_web_search(question, max_results=5):
    r = tavily.search(
        query=question,
        max_results=max_results,
        include_answer=True
    )
    answer = r.get("answer")
    results = [{"title": x.get("title"), "url": x.get("url")} for x in r.get("results", [])]
    return {"answer": answer, "results": results}


w = game_web_search("When were Pokémon Gold and Silver released?", max_results=5)
print("Answer:", w["answer"])
print("Results shown:", len(w["results"]))


Answer: Pokémon Gold and Silver were released in Japan on November 21, 1999, and in North America on October 15, 2000.
Results shown: 5


In [ ]:
def udaplay(q):
    hits  = retrieve_game(q, k=5)
    judge = evaluate_retrieval(q, hits)

    if judge.get("useful"):
        top = hits[0] if hits else {}
        ql = q.lower()
        if "platform" in ql:
            ans = f"{top.get('name')} was launched on {top.get('platform')}."
        elif "publisher" in ql:
            ans = f"The publisher of {top.get('name')} is {top.get('publisher')}."
        else:
            ans = f"{top.get('name')} ({top.get('platform')}) was released in {top.get('year')}."
        return {"mode":"local","answer":ans,"sources":hits,"eval":judge}

    web = game_web_search(q, max_results=5)
    return {"mode":"web","answer":web.get("answer") or "See sources.","sources":web.get("results",[]),"eval":judge}


### Agent

In [ ]:
import json

def make_report(question):
    r = udaplay(question)

    def show(s):
        name = s.get("name") or s.get("title") or "Unknown"
        plat = s.get("platform") or ""
        year = s.get("year") or ""
        url  = s.get("url") or ""
        parts = [name]
        if plat: parts.append(f"({plat})")
        if year: parts.append(str(year))
        if url:  parts.append(url)
        return " ".join(p for p in parts if p)

    cites = "\n".join(f"- {show(s)}" for s in r.get("sources", [])[:2])

    print("\nQ:", question)
    print("Answer:", r["answer"])
    print("Mode:", r["mode"])
    print("Sources:\n" + (cites or "- (none)"))

    payload = {
        "question": question,
        "answer": r["answer"],
        "mode": r["mode"],
        "evaluation": r.get("eval"),
        "sources": r.get("sources", []),
    }
    print("\nJSON:")
    print(json.dumps(payload, indent=2))
    return payload


In [ ]:
make_report("When were Pokémon Gold and Silver released?")
make_report("Which was the first 3D platformer Mario game?")
make_report("Was Mortal Kombat X released for PlayStation 5?")


Q: When were Pokémon Gold and Silver released?
Answer: Pokémon Gold and Silver (Game Boy Color) was released in 1999.
Mode: local
Sources:
- Pokémon Gold and Silver (Game Boy Color) 1999
- Pokémon Ruby and Sapphire (Game Boy Advance) 2002

JSON:
{
  "question": "When were Pok\u00e9mon Gold and Silver released?",
  "answer": "Pok\u00e9mon Gold and Silver (Game Boy Color) was released in 1999.",
  "mode": "local",
  "evaluation": {
    "useful": true,
    "description": "The docs provide the release year for Pok\u00e9mon Gold and Silver."
  },
  "sources": [
    {
      "name": "Pok\u00e9mon Gold and Silver",
      "platform": "Game Boy Color",
      "year": 1999,
      "publisher": "Nintendo",
      "score": 0.3122
    },
    {
      "name": "Pok\u00e9mon Ruby and Sapphire",
      "platform": "Game Boy Advance",
      "year": 2002,
      "publisher": "Nintendo",
      "score": -0.0213
    },
    {
      "name": "Wii Sports",
      "platform": "Wii",
      "year": 2006,
      "publisher

{'question': 'Was Mortal Kombat X released for PlayStation 5?',
 'answer': 'Mortal Kombat X is not an original PS5 release but a backward-compatible game playable on PS5. It was originally released for PS4 in 2015. Some PS4 features may be absent on PS5.',
 'mode': 'web',
 'evaluation': {'useful': False,
  'description': 'The documents do not mention Mortal Kombat X or its availability on PlayStation 5.'},
 'sources': [{'title': 'Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay',
   'url': 'https://www.youtube.com/watch?v=HUNMZwDdrvY'},
  {'title': 'Mortal Kombat X',
   'url': 'https://en.wikipedia.org/wiki/Mortal_Kombat_X'},
  {'title': 'Mortal Kombat X',
   'url': 'https://store.playstation.com/en-us/product/UP1018-CUSA00967_00-MORTALKOMBATX000'},
  {'title': 'Mortal Kombat X',
   'url': 'https://www.playstation.com/en-us/games/mortal-kombat-x_msm_moved/'},
  {'title': 'PlayStation 5 | Mortal Kombat X | Backwards compatible test',
   'url': 'https://www.youtube.com/watch?v=9Dto2mjnliA'}]}

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes